In [19]:
# Importando as bibliotecas
import pandas as pd
import re

In [20]:
file_path = "../data/103_B.csv"
with open(file_path, "r", encoding="utf-8") as f:
    for _ in range(31):  # primeiras 31 linhas do cabeçalho
        line = f.readline().strip()
        if ";" in line:
            # separa pelos primeiros 2 ";" para pegar Coluna A e C
            parts = line.split(";")
            key = parts[0].strip()
            value = parts[2].strip()  # Coluna C
            if key in informacoes_interessantes:
                metadados[key] = value
                
# Limpar aspas simples de todos os valores
metadados = {k: v.strip("'") for k, v in metadados.items()}

print(metadados)


# Converter START e STOP para datetime diretamente no dicionário
metadados['START'] = pd.to_datetime(metadados['START'], format="%Y/%m/%d_%H:%M:%S")
metadados['STOP'] = pd.to_datetime(metadados['STOP'], format="%Y/%m/%d_%H:%M:%S")

# Calcular duração e adicionar ao dicionário
metadados['DURATION'] = metadados['STOP'] - metadados['START']

# Mostrar resultado
for k, v in metadados.items():
    print(f"{k}: {v}")

AttributeError: 'Timedelta' object has no attribute 'strip'

In [23]:
# Caminho para o arquivo

# Lendo todo o CSV como tabela genérica
df = pd.read_csv(file_path, sep=";", header=None)

# Selecionar linhas 33 a 37 (em pandas o índice começa em 0, então é 32:37)
df_tensoes = df.loc[[32, 33, 35], [3, 4, 5]].copy()

# Nomeando colunas
df_tensoes.columns = ["V1", "V2", "V3"]

# Nomeando as linhas (conforme a ordem que você passou)
df_tensoes.index = [
    "Average value",
    "Max value",
    "Min value",
]

# Converter todas as colunas do dataframe
df_tensoes = df_tensoes.astype(str)                # garantir que são strings
df_tensoes = df_tensoes.apply(lambda x: x.str.replace(",", "."))
df_tensoes = df_tensoes.apply(pd.to_numeric, errors='coerce')

display(df_tensoes)

limite_min = 207
limite_max = 253

print(f"\nLimite mínimo: {limite_min}\nLimite máximo: {limite_max}")

tensao_max = df_tensoes.loc["Max value"]
tensao_min = df_tensoes.loc["Min value"]

for col in df_tensoes.columns[0:3]:
    if tensao_min[col] < limite_min:
        print(f"{col}: valor mínimo {tensao_min[col]} V abaixo do limite mínimo ({limite_min} V)")
    if tensao_max[col] > limite_max:
        print(f"{col}: valor máximo {tensao_max[col]} V acima do limite máximo ({limite_max} V)")
    if limite_min <= tensao_min[col] <= limite_max and limite_min <= tensao_max[col] <= limite_max:
        print(f"{col}: tensões dentro do limite.")

,V1,V2,V3
Average value,222.0,221.0,223.0
Max value,235.0,235.0,235.0
Min value,213.0,211.0,213.0



Limite mínimo: 207
Limite máximo: 253
V1: tensões dentro do limite.
V2: tensões dentro do limite.
V3: tensões dentro do limite.


In [22]:
# ---------- (re)criar df_correntes a partir do CSV, caso ainda não exista ----------
df = pd.read_csv(file_path, sep=";", header=None, dtype=str, encoding="utf-8", low_memory=False)
# linhas 33..37 -> índices 32..36; colunas M,N,O -> índices 12,13,14
df_correntes = df.loc[32:36, [12, 13, 14]].copy()
df_correntes.columns = ["A1", "A2", "A3"]
df_correntes.index = [
    "Average value",
    "Max value",
    "Max value recorded time",
    "Min value",
    "Min value recorded time",
]

# Mostrar o raw para inspeção rápida
print("Raw extraído (antes do parsing):\n")
display(df_correntes)

# ---------- Função para limpar e converter números ----------
def parse_number_cell(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    # remover aspas simples duplas
    s = s.strip("'").strip('"')
    # alguns campos vêm como '2,22E+02' (vírgula no exponencial) ou '2,22E+02 '
    # trocar vírgula por ponto, exceto nas partes não-numéricas:
    s = s.replace(',', '.')
    # manter somente caracteres válidos de float (dígitos, + - . e eE)
    s = re.sub(r"[^\d\.\-+eE]", "", s)
    if s == "":
        return np.nan
    try:
        return float(s)
    except:
        return np.nan

# ---------- Função para converter datas em formatos comuns ----------
def parse_datetime_cell(x):
    if pd.isna(x):
        return pd.NaT
    s = str(x).strip().strip("'").strip('"')
    s = s.replace('_', ' ').replace('.', ':')  # normalizar possíveis variações
    # tentar formatos específicos mais comuns
    for fmt in ("%d/%m/%Y %H:%M:%S", "%d/%m/%Y %H:%M", "%Y/%m/%d %H:%M:%S", "%Y/%m/%d %H:%M"):
        try:
            return pd.to_datetime(s, format=fmt)
        except Exception:
            pass
    # fallback robusto
    return pd.to_datetime(s, dayfirst=True, errors='coerce')

# ---------- Aplicar parsing ----------
numeric_rows = ["Average value", "Max value", "Min value"]
time_rows = ["Max value recorded time", "Min value recorded time"]

# aplicar parse_number_cell nas linhas numéricas
df_values = df_correntes.loc[numeric_rows].applymap(parse_number_cell).astype(float)
# aplicar parse_datetime_cell nas linhas de tempo
df_times = df_correntes.loc[time_rows].applymap(parse_datetime_cell)

# Exibir tipos e conteúdo
print("\n--- Valores numéricos (floats) ---")
display(df_values)
print(df_values.dtypes)

print("\n--- Timestamps (datetimes) ---")
display(df_times)
print(df_times.dtypes)

# ---------- Montar um summary (visual) combinando ambos ----------
summary = pd.concat([df_values, df_times])
# reorganizar índice de forma legível
summary = summary.loc[numeric_rows + time_rows]

print("\n--- Summary combinado (numéricos e timestamps) ---")
display(summary)

Raw extraído (antes do parsing):



,A1,A2,A3
Average value,"8,95E+00","1,28E+01","1,27E+01"
Max value,"4,80E+01","5,70E+01","4,54E+01"
Max value recorded time,20/06/2023 23:28,28/06/2023 00:05,23/06/2023 02:13
Min value,"1,68E+00","3,44E+00","3,63E+00"
Min value recorded time,25/06/2023 11:48,22/06/2023 13:22,25/06/2023 12:30



--- Valores numéricos (floats) ---


C:\Users\Ivna Façanha\AppData\Local\Temp\ipykernel_12248\3743787820.py:57: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_values = df_correntes.loc[numeric_rows].applymap(parse_number_cell).astype(float)
C:\Users\Ivna Façanha\AppData\Local\Temp\ipykernel_12248\3743787820.py:59: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_times = df_correntes.loc[time_rows].applymap(parse_datetime_cell)


,A1,A2,A3
Average value,8.95,12.80,12.70
Max value,48.00,57.00,45.40
Min value,1.68,3.44,3.63


A1    float64
A2    float64
A3    float64
dtype: object

--- Timestamps (datetimes) ---


,A1,A2,A3
Max value recorded time,2023-06-20 23:28:00,2023-06-28 00:05:00,2023-06-23 02:13:00
Min value recorded time,2023-06-25 11:48:00,2023-06-22 13:22:00,2023-06-25 12:30:00


A1    datetime64[ns]
A2    datetime64[ns]
A3    datetime64[ns]
dtype: object

--- Summary combinado (numéricos e timestamps) ---


,A1,A2,A3
Average value,8.95,12.8,12.7
Max value,48.0,57.0,45.4
Min value,1.68,3.44,3.63
Max value recorded time,2023-06-20 23:28:00,2023-06-28 00:05:00,2023-06-23 02:13:00
Min value recorded time,2023-06-25 11:48:00,2023-06-22 13:22:00,2023-06-25 12:30:00
